In [1]:
import cudaq
from cudaq import spin

In [2]:
from RBM_surrogate import *
from Sampling_Quantum import *

In [3]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [4]:
N=4
M=4
D=0
beta=1

g=0.5

In [5]:
def TFIM(N,g):
    hamiltonian = 0
    for i in range(N-1):
        hamiltonian -= spin.z(i) * spin.z(i+1)

    for i in range(N):
        hamiltonian -= g*spin.x(i)
    return hamiltonian

In [6]:
Ham = TFIM(N,g)
print(Ham.term_count)

8


In [7]:
# Adam optimizer hyperparameters
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

In [8]:
from scipy.sparse import csr_array, coo_matrix
from scipy.sparse.linalg import eigsh

Ham_sp = Ham.to_sparse_matrix()
data, row, col = Ham_sp
Ham_coo = coo_matrix((data, (row, col)), shape=(2**8, 2**8))
Ham_csr = csr_array(Ham_coo)

lmbd, _ = eigsh(Ham_csr, k=1, which='SA')

print("Ground State:", lmbd)

Ground State: [-3.42703409]


In [9]:
def plot_log(E_hist, E_smpl_hist):
    #ya, iter = smpl.median_filter(np.real(E_hist))
    ya = np.real(E_hist)
    xa = np.arange(len(ya))
    ci = np.abs(np.array(E_smpl_hist) - ya)
    fig, ax = plt.subplots()
    ax.plot(ya)
    ax.fill_between(xa, (ya-ci), (ya+ci), color='r', alpha=.5)
    ax.set_ylabel("Energy")
    ax.set_xlabel("Epochs")
    plt.hlines(lmbd[0],0,len(xa)-1,color='r',linestyles='dashed')
    plt.show()

In [10]:
import tensorflow as tf

2025-11-05 14:04:00.819766: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 14:04:00.841960: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-05 14:04:00.841982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-05 14:04:00.842561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-05 14:04:00.846415: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
from functools import partial

sample_size = 20000

seed = 1

E_hist=[]
E_smpl_hist=[]

rbm = RBM_surrogate(N=N,M=M,seed=seed,)
x = tf.Variable(rbm.X)

tm=time.time()

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=epsilon)

for epoch in range(40):
  rbm = RBM_surrogate(N,M,X=x.numpy())
  rbm.build_surrogate()

  tm = time.time()
  # prob_func = partial(prob_Ising_nv, N=N, poly=rbm.poly, log_rho_max=rbm.log_rho_max)
  prob_dict = Sampling_Quantum(N=N, poly=rbm.poly, sample_size=sample_size, tot_time=12, time_delta=0.5, gamma=0.42, beta=1, burn=None, init_config=[], compute_proposal_matrix=False)
  #prob_dist = Sampling_Quantum(N=N, prob_func=prob_func, sample_size=sample_size, burn=sample_size//10)
  print("Sampling Time: ", time.time()-tm)

  E = rbm.Energy_exact(Ham)

  #tm = time.time()
  E_smpl = rbm.Energy_sampling(Ham,prob_dict=prob_dict)
  #print("Energy Time: ", time.time()-tm)
  print(epoch, "Energy: ", E, "\t +/- \t", np.abs(E_smpl - E))

  #tm = time.time()
  #grad_exact = np.real(rbm.grad_exact(Ham))
  grad = np.real(rbm.grad_Sampling(Ham,prob_dict=prob_dict))
  #print("Gradient Time: ", time.time()-tm)

  optimizer.apply_gradients([(grad, x)])

  E_hist.append(E)
  E_smpl_hist.append(E_smpl)

print("\n\n\nEnergy: ",E,"\n Time",time.time()-tm, lmbd[0])

plot_log(E_hist, E_smpl_hist)

2025-11-05 14:04:01.890097: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-05 14:04:01.890363: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Sampling Time:  43.085930585861206
0 Energy:  -1.925650904411052 	 +/- 	 0.01985032470845005
Sampling Time:  39.79267430305481
1 Energy:  -2.0116275332261346 	 +/- 	 0.034537793528626626
